In [1]:
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain.schema.runnable import RunnableLambda, RunnableBranch, RunnableSequence
import os
from dotenv import load_dotenv
load_dotenv()
from langchain_groq import ChatGroq


groq_api_key=os.getenv("GROQ_API_KEY")

model=ChatGroq(groq_api_key=groq_api_key,model_name="qwen-2.5-32b")

In [2]:
from pydantic import BaseModel, Field
from typing import Literal
from langchain_core.output_parsers import PydanticOutputParser

In [3]:
class StuOutputCorrectness(BaseModel):
    sentiment: Literal['correct', 'partially_correct', 'incorrect'] = Field(description='Give the sentiment from the student response')
    
parser = PydanticOutputParser(pydantic_object=StuOutputCorrectness)

In [4]:
prompt = PromptTemplate(
    template=(
        "Analyze the following student response and classify its correctness. "
        "Determine whether it is 'correct', 'partially_correct', or 'incorrect'.\n\n"
        "Student Response: {response}\n\n"
        "Provide the classification following this format:\n{format_instruction}"
    ),
    input_variables=['response'],
    partial_variables={
        'format_instruction': parser.get_format_instructions()
    }
)


In [5]:
classifier_chain = prompt | model | parser

In [12]:
prompt_correct = PromptTemplate(
    template=(
        "The student provided a correct response: {response}.\n\n"
        "Generate a focused and concise adaptive prompt to guide the student to the next step "
        "without unnecessary repetition. The adaptive prompt should ensure that the model only provides "
        "the best next-step guidance in solving the problem."
    ),
    input_variables=['response']
)

prompt_partially_correct = PromptTemplate(
    template=(
        "The student's response is partially correct: {response}.\n\n"
        "Identify the specific mistake or misunderstanding in the response and provide constructive feedback. "
        "Offer guidance that helps the student recognize their mistake and encourage them to refine their answer. "
        "Additionally, include a strategic hint or ask the student: 'Would you like more hints?' to ensure engagement.\n\n"
        "Now, generate the best adaptive prompt so the model can provide the most effective response for the next step."
    ),
    input_variables=['response']
)

prompt_incorrect = PromptTemplate(
    template=(
        "The student's response is incorrect: {response}.\n\n"
        "Analyze why the student may have made this mistake. Did they misunderstand the concept? "
        "Did they miscalculate? Provide targeted clarification based on the likely issue. "
        "Offer a step-by-step hint that gradually leads them toward the correct answer.\n\n"
        "Now, generate the only one best adaptive prompt that ensures the model provides an effective response to help the student progress."
    ),
    input_variables=['response']
)


In [13]:
str_parser = StrOutputParser()

In [14]:
branch_chain = RunnableBranch(
    (lambda x: x.sentiment == 'correct', prompt_correct | model | str_parser),
    (lambda x: x.sentiment == 'partially_correct', prompt_partially_correct | model | str_parser),
    (lambda x: x.sentiment == 'incorrect', prompt_incorrect | model | str_parser),
    RunnableLambda(lambda x: "Could find any kind of sentiment.")
)

In [15]:
final_chain = classifier_chain | branch_chain

In [16]:
adaptive_prompt = final_chain.invoke(
    {
        'response': "the sum of 2+2=5",
    }
)

In [17]:
print(adaptive_prompt)

To create an adaptive prompt that ensures the model provides an effective response to help the student progress, we need to focus on what information is missing or misunderstood by the student. Since the specific context of the student's mistake is not provided, I'll design a prompt that can be broadly applied to various learning scenarios where a student has answered incorrectly due to a possible misunderstanding or miscalculation.

Adaptive Prompt:
"First, let's identify where the misunderstanding or miscalculation occurred. Can you walk through the steps you took to reach your answer? Once we pinpoint the exact step where things went awry, I'll provide a targeted explanation or clarification. This will help you understand the concept more clearly and avoid similar mistakes in the future. Remember, the goal is to learn from this, not to just get the right answer. Let's break it down together."

This prompt is designed to engage the student in self-reflection and to identify the root 